In [3]:
# %load_ext autoreload
# %autoreload 2
import podpac
import podpac.datalib
import numpy as np

In [4]:
username=None
password=None
smap = podpac.datalib.smap.SMAP(product='SPL3SMP', username=username, password=password)

## SMAP source example

In [5]:
smap.source_coordinates

Coordinates (EPSG:4326)
	time: ArrayCoordinates1d(time): Bounds[2015-03-31, 2020-04-20], N[1810]

In [6]:
nc = smap.find_coordinates()[0]
nc

INFO:pydap:Fetching URL: https://n5eil02u.ecs.nsidc.org/opendap/SMAP/SPL3SMP.006/2015.03.31/SMAP_L3_SM_P_20150331_R16510_001.h5.dods?Soil_Moisture_Retrieval_Data_AM_longitude[0:1:405][0:1:963]
INFO:pydap:Fetching URL: https://n5eil02u.ecs.nsidc.org/opendap/SMAP/SPL3SMP.006/2015.03.31/SMAP_L3_SM_P_20150331_R16510_001.h5.dods?Soil_Moisture_Retrieval_Data_AM_latitude[0:1:405][0:1:963]


Coordinates (EPSG:4326)
	time: ArrayCoordinates1d(time): Bounds[2015-03-31, 2020-04-20], N[1810]
	lat: ArrayCoordinates1d(lat): Bounds[-83.63196563720703, 83.63196563720703], N[406]
	lon: ArrayCoordinates1d(lon): Bounds[-179.81292724609375, 179.81292724609375], N[964]

In [7]:
c = podpac.coordinates.merge_dims([nc.drop('time'), podpac.Coordinates(['2018-05-19T12'], ['time'])])
c

Coordinates (EPSG:4326)
	lat: ArrayCoordinates1d(lat): Bounds[-83.63196563720703, 83.63196563720703], N[406]
	lon: ArrayCoordinates1d(lon): Bounds[-179.81292724609375, 179.81292724609375], N[964]
	time: ArrayCoordinates1d(time): Bounds[2018-05-19T12, 2018-05-19T12], N[1]

In [8]:
smap.source_coordinates.intersect(c, outer=True)

Coordinates (EPSG:4326)
	time: ArrayCoordinates1d(time): Bounds[2018-05-19, 2018-05-19], N[1]

In [9]:
smap.interpolation

In [10]:
out = smap.eval(c)  # <-- returns all nans
out.data

INFO:pydap:Fetching URL: https://n5eil02u.ecs.nsidc.org/opendap/SMAP/SPL3SMP.006/2018.05.19/SMAP_L3_SM_P_20180519_R16510_001.h5.dods?Soil_Moisture_Retrieval_Data_AM_soil_moisture[0:1:405][0:1:963]
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


array([[[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       ...,

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]]])

In [11]:
np.where(np.isnan(out.data))

(array([  0,   0,   0, ..., 405, 405, 405], dtype=int64),
 array([  0,   1,   2, ..., 961, 962, 963], dtype=int64),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int64))

## Recreate Simple Example

In [16]:
from podpac.coordinates import Coordinates, clinspace, UniformCoordinates1d
import numpy as np

a = Coordinates([clinspace('2015-03-31', '2018-06-03', 1161)], ['time'])
b = Coordinates([clinspace(-83.63197326660156, 83.63196563720703, 406), 
                 clinspace(-179.8129119873047, 179.8129119873047, 964), 
                 '2018-05-19T12'], ['lat', 'lon','time'])

c = Coordinates([clinspace(-83.63197326660156, 83.63196563720703, 406), 
                 clinspace(-179.8129119873047, 179.8129119873047, 964), 
                 clinspace('2015-05-19T12', '2018-05-20T20', 1647)], ['lat', 'lon','time'])

d = Coordinates([clinspace(0, 10, 11)], ['lat'])
e = Coordinates([[5.2]], ['lat'])

a.intersect(b, outer=True)
# d.intersect(e)

Coordinates (EPSG:4326)
	time: UniformCoordinates1d(time): Bounds[2018-05-19, 2018-05-19], N[1]

In [19]:
self = a['time']
bounds = b['time'].bounds
outer = True

lo = max(bounds[0], self.bounds[0])
hi = min(bounds[1], self.bounds[1])

imin = int(np.ceil((lo - self.bounds[0]) / np.abs(self.step)))
imax = int(np.floor((hi - self.bounds[0]) / np.abs(self.step)))

if outer:
    imin -= 1
    imax += 1

imax = np.clip(imax+1, 0, self.size)
imin = np.clip(imin, 0, self.size)

# empty case
if imin >= imax:
    ret = self._select_empty(return_indices)

if self.is_descending:
    imax, imin = self.size - imin, self.size - imax

start = self.start + imin*self.step
stop = self.start + (imax-1)*self.step
c = UniformCoordinates1d(start, stop, self.step, **self.properties)

In [20]:
bounds

(numpy.datetime64('2018-05-19T12','h'), numpy.datetime64('2018-05-19T12','h'))

In [21]:
a = Coordinates([['2015-05-19', '2018-05-20']], ['time'])
b = Coordinates([clinspace(0, 10, 11), 
                 clinspace(0, 10, 11), 
                 '2018-05-19T12'], ['lat', 'lon','time'])

a.intersect(b, outer=True)
# d.intersect(e)

Coordinates (EPSG:4326)
	time: ArrayCoordinates1d(time): Bounds[2015-05-19, 2018-05-20], N[2]

In [22]:
self = a['time']
bounds = b['time'].bounds

In [25]:
gt = np.where(self.coords['time'] >= bounds[0])[0]
lt = np.where(self.coords['time'] <= bounds[1])[0]

In [26]:
gt[0]

1

In [27]:
lt[0]

0